In [18]:
import json
import pandas as pd
import numpy as np
import csv
import random
import matplotlib.pyplot as plt
import seaborn as sns
from joypy import joyplot
from matplotlib.cm import viridis
import plotly.express as px
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset

In [19]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("asaniczka/tmdb-movies-dataset-2023-930k-movies")

# print("Path to dataset files:", path)


# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("aradhanahirapara/product-retail-price-survey-2017-2025")

# print("Path to dataset files:", path)

In [20]:
Data = pd.read_csv(r'E:\Group Data Project\Data-Science\Source\Data\Retail_Prices_of _Products.csv')
DF = pd.DataFrame(Data)
DF

,Year,Month,GEO,Product Category,Products,VALUE,Taxable,Total tax rate,Value after tax,Essential,COORDINATE,UOM
0,2017,January,Province 1,Meat & Poultry,"Beef stewing cuts, per kilogram",12.66,No,11.0,12.66,Essential,11.100,Dollars
1,2017,January,Province 1,Meat & Poultry,"Beef striploin cuts, per kilogram",21.94,No,11.0,21.94,Essential,11.200,Dollars
2,2017,January,Province 1,Meat & Poultry,"Beef top sirloin cuts, per kilogram",13.44,No,11.0,13.44,Essential,11.300,Dollars
3,2017,January,Province 1,Meat & Poultry,"Beef rib cuts, per kilogram",20.17,No,11.0,20.17,Essential,11.410,Dollars
4,2017,January,Province 1,Meat & Poultry,"Ground beef, per kilogram",9.12,No,11.0,9.12,Essential,11.400,Dollars
...,...,...,...,...,...,...,...,...,...,...,...,...
118477,2025,February,Province 11,Nuts & Snacks,"Sunflower seeds, 400 grams",4.50,No,12.0,4.50,Non-Essential,10.109,Dollars
118478,2025,February,Province 11,Toiletries & Cleaning,"Deodorant, 85 grams",8.07,Yes,12.0,9.04,Essential,10.750,Dollars
118479,2025,February,Province 11,Toiletries & Cleaning,"Toothpaste, 100 millilitres",4.13,Yes,12.0,4.63,Essential,10.760,Dollars
118480,2025,February,Province 11,Toiletries & Cleaning,"Shampoo, 400 millilitres",7.18,Yes,12.0,8.04,Essential,10.770,Dollars


In [21]:
DF.describe()

,Year,VALUE,Total tax rate,Value after tax,COORDINATE
count,118482.000000,118482.000000,118482.000000,118482.000000,118482.000000
mean,2020.591837,5.437324,12.632134,5.515499,6.501741
std,2.355363,5.192748,2.899124,5.215311,3.172173
min,2017.000000,0.300000,5.000000,0.300000,1.100000
25%,2019.000000,2.850000,11.000000,2.910000,3.750000
50%,2021.000000,3.990000,13.000000,4.050000,6.500000
75%,2023.000000,5.810000,15.000000,5.947500,9.240000
max,2025.000000,48.450000,15.000000,48.450000,11.990000


In [22]:
DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118482 entries, 0 to 118481
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Year              118482 non-null  int64  
 1   Month             118482 non-null  object 
 2   GEO               118482 non-null  object 
 3   Product Category  118482 non-null  object 
 4   Products          118482 non-null  object 
 5   VALUE             118482 non-null  float64
 6   Taxable           118482 non-null  object 
 7   Total tax rate    118482 non-null  float64
 8   Value after tax   118482 non-null  float64
 9   Essential         118482 non-null  object 
 10  COORDINATE        118482 non-null  float64
 11  UOM               118482 non-null  object 
dtypes: float64(4), int64(1), object(7)
memory usage: 10.8+ MB


In [23]:
null_columns = DF.columns[DF.isnull().any()]
print("Cac cot co gia tri NULL:", list(null_columns))

Cac cot co gia tri NULL: []


In [24]:
DF.reset_index(level=0, inplace=True)
DF

,index,Year,Month,GEO,Product Category,Products,VALUE,Taxable,Total tax rate,Value after tax,Essential,COORDINATE,UOM
0,0,2017,January,Province 1,Meat & Poultry,"Beef stewing cuts, per kilogram",12.66,No,11.0,12.66,Essential,11.100,Dollars
1,1,2017,January,Province 1,Meat & Poultry,"Beef striploin cuts, per kilogram",21.94,No,11.0,21.94,Essential,11.200,Dollars
2,2,2017,January,Province 1,Meat & Poultry,"Beef top sirloin cuts, per kilogram",13.44,No,11.0,13.44,Essential,11.300,Dollars
3,3,2017,January,Province 1,Meat & Poultry,"Beef rib cuts, per kilogram",20.17,No,11.0,20.17,Essential,11.410,Dollars
4,4,2017,January,Province 1,Meat & Poultry,"Ground beef, per kilogram",9.12,No,11.0,9.12,Essential,11.400,Dollars
...,...,...,...,...,...,...,...,...,...,...,...,...,...
118477,118477,2025,February,Province 11,Nuts & Snacks,"Sunflower seeds, 400 grams",4.50,No,12.0,4.50,Non-Essential,10.109,Dollars
118478,118478,2025,February,Province 11,Toiletries & Cleaning,"Deodorant, 85 grams",8.07,Yes,12.0,9.04,Essential,10.750,Dollars
118479,118479,2025,February,Province 11,Toiletries & Cleaning,"Toothpaste, 100 millilitres",4.13,Yes,12.0,4.63,Essential,10.760,Dollars
118480,118480,2025,February,Province 11,Toiletries & Cleaning,"Shampoo, 400 millilitres",7.18,Yes,12.0,8.04,Essential,10.770,Dollars


--------------------------------------------------------------------------------------------DIEN GIA TRI CON THIEU------------------------------------------------------------------------------------------------

2017 -> 20177 (5 values)

In [25]:
DF.loc[DF['index'].between(0,4), 'Year'] = 20177
DF

,index,Year,Month,GEO,Product Category,Products,VALUE,Taxable,Total tax rate,Value after tax,Essential,COORDINATE,UOM
0,0,20177,January,Province 1,Meat & Poultry,"Beef stewing cuts, per kilogram",12.66,No,11.0,12.66,Essential,11.100,Dollars
1,1,20177,January,Province 1,Meat & Poultry,"Beef striploin cuts, per kilogram",21.94,No,11.0,21.94,Essential,11.200,Dollars
2,2,20177,January,Province 1,Meat & Poultry,"Beef top sirloin cuts, per kilogram",13.44,No,11.0,13.44,Essential,11.300,Dollars
3,3,20177,January,Province 1,Meat & Poultry,"Beef rib cuts, per kilogram",20.17,No,11.0,20.17,Essential,11.410,Dollars
4,4,20177,January,Province 1,Meat & Poultry,"Ground beef, per kilogram",9.12,No,11.0,9.12,Essential,11.400,Dollars
...,...,...,...,...,...,...,...,...,...,...,...,...,...
118477,118477,2025,February,Province 11,Nuts & Snacks,"Sunflower seeds, 400 grams",4.50,No,12.0,4.50,Non-Essential,10.109,Dollars
118478,118478,2025,February,Province 11,Toiletries & Cleaning,"Deodorant, 85 grams",8.07,Yes,12.0,9.04,Essential,10.750,Dollars
118479,118479,2025,February,Province 11,Toiletries & Cleaning,"Toothpaste, 100 millilitres",4.13,Yes,12.0,4.63,Essential,10.760,Dollars
118480,118480,2025,February,Province 11,Toiletries & Cleaning,"Shampoo, 400 millilitres",7.18,Yes,12.0,8.04,Essential,10.770,Dollars


Sua loi

2017 -> 217 (5 values)

In [ ]:
DF.loc[DF['index'].between(5,9), 'Year'] = 217
DF

Sua loi

January -> Jan (5 values)

In [ ]:
DF.loc[DF['index'].between(10,14), 'Month'] = 'Jan'
DF

Sua loi

January -> january (5 values)

In [ ]:
DF.loc[DF['index'].between(15,19), 'Month'] = 'january'
DF

Sua loi

January -> 1 (5 values)

In [ ]:
DF.loc[DF['index'].between(20,24), 'Month'] = 1
DF

Sua loi

Province 1 -> Province1 (5 values)

In [ ]:
DF.loc[DF['index'].between(25,29), 'GEO'] = 'Province1'
DF

Sua loi

Province 1 -> province1 (5 values)

In [ ]:
DF.loc[DF['index'].between(30,34), 'GEO'] = 'province1'
DF

Sua loi

Columns VALUE -> NULL (5 values)

In [ ]:
DF.loc[DF['index'].between(35,39), 'VALUE'] = np.nan
DF

Sua loi

Columns total tax rate -> .000 (5 values)

In [ ]:
DF.loc[DF['index'].between(40,44), 'Total tax rate'] = np.nan
DF

Sua loi

Columns values after tax -> NULL (5 values)

In [ ]:
DF.loc[DF['index'].between(45,49), 'Value after tax'] = np.nan
DF

Sua loi

Columns COORDINATE -> NULL (5 values)

In [ ]:
DF.loc[DF['index'].between(50,54), 'COORDINATE'] = np.nan
DF

Sua loi

--------------------------------------------------------------------------------------------DIEN GIA TRI CON THIEU------------------------------------------------------------------------------------------------

--------------------------------------------------------------------------------------------LAM MIN DU LIEU ------------------------------------------------------------------------------------------------

--------------------------------------------------------------------------------------------LAM MIN DU LIEU ------------------------------------------------------------------------------------------------